<a href="https://colab.research.google.com/github/lucatombesi99/Bayesian_Persuasion_Uncertainty/blob/main/Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get install cython3 libglpk-dev python3 python3.9-dev python3-pip python3-scipy

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3 is already the newest version (3.10.6-1~22.04).
python3 set to manually installed.
The following additional packages will be installed:
  libamd2 libbtf1 libcamd2 libccolamd2 libcholmod3 libcolamd2 libcxsparse3
  libglpk40 libgmp-dev libgmpxx4ldbl libgraphblas-dev libgraphblas6
  libimagequant0 libklu1 liblbfgsb0 libldl2 libmetis5 libmongoose2
  libpython3.9 libpython3.9-dev libpython3.9-minimal libpython3.9-stdlib
  libraqm0 librbio2 libsliplu1 libspqr2 libsuitesparse-dev
  libsuitesparseconfig5 libumfpack5 libxsimd-dev mailcap mime-support
  python3-beniget python3-decorator python3-gast python3-numpy python3-olefile
  python3-pil python3-ply python3-pythran python3-setuptools python3-wheel
  python3.9 python3.9-minimal
Suggested packages:
  cython-doc libiodbc2-dev gmp-doc libgmp10-doc libmpfr-dev libxsimd-doc
  python-numpy-doc python3-pytest python-pil-doc python-ply-doc
  pyth

In [ ]:
!pip install pypoman

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.3/159.3 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pycddlib: filename=pycddlib-2.1.7-cp310-cp310-linux_x86_64.whl size=866243 sha256=1791ec2a3e80e642a2c5a6c83f8afa957a435974b53f23ce79f4ae41c64e55bb
  Stored in directory: /root/.cache/pip/wheels/a2/cd/dd/04d24f3292a1860c7b82591d4e59df9246b95f5b5490e1047d
Successfully built pycddlib


In [ ]:
!pip install pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 29.2 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pypoman
from pulp import *



#setting

theta = 3
actions = 3
delta = 0.2
actions_tot = np.arange(0, actions)
actions_tot = actions_tot.tolist()
prior = np.random.random(theta)
prior = prior / np.sum(prior)
for i in range(theta):
  prior[i] = round(prior[i],8)
# Create two matrices with random values
u_r = np.random.random((theta, actions))  # Random values between 0 and 1
u_s = np.random.random((theta, actions))  # Random values from a standard normal distribution
for i in range(theta):
  for j in range(actions):
    u_r[i,j]=round(u_r[i,j],2)
    u_s[i,j]=round(u_s[i,j],2)

# Printing the matrices
print("Receiver utility matrix, (rows are theta values,cols are actions):")
print(u_r)
print("\n""Sender utility matrix, (rows are theta values,cols are actions):")
print(u_s)


Receiver utility matrix, (rows are theta values,cols are actions):
[[0.67 0.11 0.14]
 [0.08 0.24 0.26]
 [0.51 0.48 0.9 ]]

Sender utility matrix, (rows are theta values,cols are actions):
[[0.26 1.   0.89]
 [0.46 0.64 0.46]
 [0.33 0.32 0.77]]


In [ ]:
prior

array([0.00453502, 0.35936175, 0.63610323])

In [ ]:
#function to find all possible partitions
def partitions(arr):
    def generate_partitions(arr, i, current_partition, all_partitions):
        if i == len(arr):
            all_partitions.append(current_partition[:])  # Append a copy of the current partition
            return

        # Include the current element in the current partition
        current_partition.append(arr[i])
        generate_partitions(arr, i + 1, current_partition, all_partitions)
        current_partition.pop()  # Backtrack by removing the last element

        # Start a new partition with the current element
        current_partition.append([arr[i]])
        generate_partitions(arr, i + 1, current_partition, all_partitions)
        current_partition.pop()  # Backtrack by removing the last partition

    all_partitions = []
    generate_partitions(arr, 0, [], all_partitions)
    return all_partitions

In [ ]:
#creation of all possible partitions
final_values = []
final_x = []
final_v = []

result = partitions(actions_tot)
#separation of a single partition
for i in range(len(result)):
    br = [element for element in result[i] if isinstance(element, int)]
    not_br = np.array([x for x in actions_tot if x not in br])
    vertices_tot = []
    utilities_sender_from_vertices = []
    #print(br)
    #print(not_br)
    m = len(not_br)
    n = len(br)

    if n > 0:
      # vertex search for every action belongin to the br delta set,valuating each action as the a* action
      for g in range(n):

          matrix_a = np.zeros((m+2*n+theta,theta))
          matrix_b = np.zeros((m+2*n+theta))
          a_star = br[g]
          br_temp = [x for x in br if x != a_star]
          #for every state of nature
          for j in range(theta):
            #constraint 1 of the algorithm
            for k in range(m):
              matrix_a[k,j] = (u_r[j,not_br[k]]-u_r[j,a_star])
              matrix_b[k] = -delta
            #constraint 2 e 3 of the algorithm
            for i in range(len(br_temp)):
              matrix_a[m+i,j] = (u_r[j,br_temp[i]]-u_r[j,a_star])
              matrix_b[m+i] = 0
              matrix_a[m+n-1+i,j] = (u_r[j,a_star]-u_r[j,br_temp[i]])
              matrix_b[m+n-1+i] = delta
            #constraints to guarantee a valid posterior
            matrix_a[m+n-1+n-1,j] = 1
            matrix_b[m+n-1+n-1] = 1
            matrix_a[m+n-1+n-1+1,j] = -1
            matrix_b[m+n-1+n-1+1] = -1
            for t in range(theta):
              matrix_a[m+n-1+n-1+2+t,t] = -1
              matrix_b[m+n-1+n-1+2+t] = 0
          #print(matrix_a)
          #print(matrix_b)
          #vertex search
          vertices = pypoman.compute_polytope_vertices(matrix_a,matrix_b)
          #print(vertices)
          for i in vertices:
            #correct approximation of the posteriors
            for j in range(theta):
                i[j] = round(i[j],8)
                i[j] = abs(i[j])
            if np.sum(i)>1:
                i[0]=i[0]-0.00000001

          #computing sender utility and consider the one with the worst sender utility
          for i in vertices:
            u_sender_min = np.sum(u_s[:,br[0]]*i)
            #print(u_sender_min)
            for j in range(len(br)):
              utility = np.sum(u_s[:,br[j]]*i)
              #print(utility)
              if u_sender_min > utility:
                u_sender_min = utility
            u_sender_min = round(u_sender_min,8)
            utilities_sender_from_vertices.append(u_sender_min)

            vertices_tot.append(i)

      #elimination of the duplicate vertices breaking ties in favour of the sender
      final_vertices = set(tuple(arr) for arr in vertices_tot if arr.size > 0)
      final_vertices = list(final_vertices)
      final_utilities = np.full((len(final_vertices)),-np.inf)
      for i in range(len(vertices_tot)):
        for j in range(len(final_vertices)):
          if (vertices_tot[i] == final_vertices[j]).all() and utilities_sender_from_vertices[i] > final_utilities[j]:
            final_utilities[j] = utilities_sender_from_vertices[i]

      #print("final vertices")
      #print(final_vertices)
      #print("final utilities")
      print(final_utilities)
      #give a probability to every vertex and maximize the Sender utility associated to every vertex
      if len(final_vertices) > 0:
        Lp_prob = pulp.LpProblem('Problem', LpMaximize)
        final_vertices = list(final_vertices)
        dim = range(0,len(final_vertices))

        #print(final_vertices[0])
        #print(prior)
        #signaling scheme
        x = pulp.LpVariable.dicts('x',(dim), lowBound=0)


        #obj. function
        Lp_prob += pulp.lpSum([x[i]*final_utilities[i] for i in dim])

        for i in dim:
          Lp_prob += x[i] >= 0


        #constraints
        Lp_prob += pulp.lpSum(x[j] for j in dim) == 1


        for i in range(theta):
          Lp_prob += pulp.lpSum(x[j]*final_vertices[j][i] for j in dim) == prior[i]

        #print("LP Problem formulation:")
        #print(Lp_prob)

        Lp_prob.solve()

        if  LpStatus[Lp_prob.status]=="Optimal":
          print("possible solution found")
          X = []
          final_value = value(Lp_prob.objective)
          #print(final_value)
          for v in Lp_prob.variables():
            if v != None:
              X.append(v.varValue)
          final_values.append(final_value)
          final_x.append(X)
          #print(final_x)
          final_v.append(final_vertices)


[0, 1, 2]
[]
final vertices
[(0.10415938, 0.4484446, 0.44739602), (0.0, 1.0, 0.0), (0.0, 0.9047619, 0.0952381), (0.38378189, 0.17581265, 0.44040545), (0.25352113, 0.74647887, 0.0), (0.5, 0.5, 0.0)]
[0.38100664 0.46       0.44761905 0.32599091 0.40929577 0.36      ]
[0, 1]
[2]
final vertices
[]
[]
[0, 2]
[1]
final vertices
[(0.10415938, 0.4484446, 0.44739602), (0.5, 0.5, 0.0), (0.38378189, 0.17581265, 0.44040545), (0.20652174, 0.0, 0.79347826), (0.53521127, 0.46478873, 0.0), (0.42391304, 0.0, 0.57608696), (0.64130435, 0.0, 0.35869565)]
[0.38100664 0.36       0.32599091 0.31554348 0.35295775 0.30032609
 0.2851087 ]
[0]
[1 2]
final vertices
[(1.0, 0.0, 0.0), (0.64130435, 0.0, 0.35869565), (0.53521127, 0.46478873, 0.0)]
[0.26       0.2851087  0.35295775]
[1, 2]
[0]
final vertices
[(0.10415938, 0.4484446, 0.44739602), (0.0, 0.9047619, 0.0952381), (0.0, 0.55, 0.45)]
[0.53433065 0.48952381 0.496     ]
[1]
[0 2]
final vertices
[]
[]
[2]
[0 1]
final vertices
[(0.10415938, 0.4484446, 0.44739602)

In [ ]:
#find the startegy that guarantees the maimum Sender utility
#print(final_values)
optimum_value = max(final_values)
index_of_maximum = final_values.index(optimum_value)
X = final_x[index_of_maximum]



#compute robust and optimum signaling scheme
phi_star = np.zeros([len(X),theta])
for i in range(theta):
  for j in range(len(X)):
    phi_star[j,i] = X[j]*vertices[j][i]/prior[i]

print(phi_star) #robust and optimum signaling scheme

[0.6591420630034248]
[[0.         0.         0.        ]
 [0.99999999 0.         0.02739182]
 [0.         0.         0.51038251]
 [0.         1.         0.46222568]]
